# Food Carts and BARTs

## Introduction/Business Problem- Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. 

#### A description of the problem and a discussion of the background:

#### Who and what? About the business:  In San Francisco, an enterprising entrepreneur is launching new food cart service, centered around snacks and non-alcoholic beverages. He envisions starting with five food carts to distribute the company's presence to make a splash effect when it enters the market.  He wants to place them at high volume locations at peak times and may be open to placing one at multiple locations, or a couple at several locations, or some combination thereof.  He wants to sell to BART (BART, essentially the San Francisco metro, is an acronym for Bay Area Rapid Transit) commuters who are too busy to have gotten their beverage and/or snack at home, or are beverage and/or snack enthusiasts.  He wants to make efficient use of his limited food cart resources to ensure the project springs from the get-go.

#### Why this study? What next?:  This preliminary study is to provide a foundation for a future boots-on-the-ground survey of commuters to gauge project interest, which will ask whether or not the person drinks beverages (hot/cold) around that time of day, and whether or not they already had that beverage at home or outside.  This survey is out of the scope of this project, but its description serves to provide context for this project.

#### Where? BART research to do: There are four BART lines that run at regular intervals (in both directions) through the main San Francisco stops.  I'll need to determine the number of trains per hour running through a given station to determine times to analyze location data around the BART station coordinates. There are 9 main stops to investigate in the San Francisco area, titled Daly City, Balboa Park, and Glen Park, 24th St Mission, 16th St Mission, Civic Center/UN Plaza, Powell St., Montgomery St, and Embarcadero.  In geographic order, Daly City is the first stop and Embarcadero is the last stop in the downtown Financial District.  Perhaps ridership data is publicly available to give insights about the most active stations. Comparisons between the data sets could be made about different concepts of what constitutes a "good" station.

#### When? Timing research:  I hypothesize that finding a window in which the consumer has enough time to order and receive their request prior to boarding the BART is crucial.  I'll tentatively look at location/activity data times 10 minutes prior to train arrival.  If train arrival instance numbers are too high, remodeling may be needed to look only at activity over larger time chunks. Regardless, comparing specific time blocks with large time ranges will be important in determining how long mobile food carts will remain at a given location, so both scopes of data will be valuable.  I hypothesize that 06:30 - 08:30 will be especially active, as well as 11:30 - 13:30 and 16:30 - 18:30, as those are common time ranges of movement (also common mealtimes). I hypothesize that lunchtime will be especially active in the main four downtown locations, as there is a dense concentration of the workforce in those areas.  I also predict a wave of activity starting in the peripheral (outside of downtown) stations in the morning and moving towards downtown, and a reciprocal wave in the evening, starting at downtown stations and moving towards the periphery, due to assumed commute patterns and residential and urban geography.

#### How?: To obtain benchmarks for high activity times to target future survey participants, it will be important to confirm times during which cafes and/or coffee shops have the highest foot traffic around BART stations (Foursquare data), as well as other data. More on this below!

## Data Section- A description of the data and how it will be used to solve the problem.

#### I'll run Foursquare API “search” searches for specific venue categories to obtain the total number of businesses in the “Cafe” and “Coffee Shop” categories within the 250 meter radius of each BART station (determined by latitudes and longitudes of the stations).  I hypothesize this data will be helpful for at least two purposes 1) to indicate likelihood the people buy coffee/snacks in a given area and 2) to get an understanding of the type and intensity of the competition in a given area.
#### I'll also use this "search" data to calculate the mean frequency of occurrence of each categories and include the Cafe and Coffee Shop frequencies as data qualifiers for BART stations to give an idea of the proportion of the coffee/cafe points relative to other businesses around the BART stations.  I hypothesize this data will be valuable to the entrepreneur to understand the density of the competition in a given area.
#### Preliminary research has shown there are too many trains per hour to select time segments for study based on train arrival, therefore 20 minute timeframes have been chosen for time segmentation.  I'll run Foursquare “explore” and “trending” searches in 20 minute timeframes from 6:30 - 8:30 AM to see which BART stations have high engagement (i.e. Foursquare foot traffic) with cafes/coffee shops. To determine "high engagement," at least one of the top three trending sites must be either Cafe or Coffee Shop; I'll manually record this data (logged by True or False tallies) to associate it with each BART station. There will thus be 6 timeframes: 6:30 - 6:50; 6:51 - 7:10; 7:11 - 7:30; 7:31 - 7:50; 7:51 - 8:10; 8:11 - 8:30. I hypothesize this data will be helpful to distinguish similar stations and specify which are more "engaging."
#### The station with the most True trending 20 minute time periods and with the most Cafe and Coffee shop locations will be recognized as the station with the highest potential/"most engaging" (assuming the entrepreneur's product is the best available and will squash all competition;  should the entrepreneur's product not be the best, frequency of cafe/coffee shops can be used for determining placement strategy amongst competition). 
#### K-means clustering will be used to figure out how to segment the potential sites for the five food carts, as the geographic separation will ensure distribution of the product as well as the brand. This clustering separation will facilitate further analysis to see the potential value of having multiple carts in individual clusters. 
#### In summary, each individual BART station will have the following components attributed to them based on their 250 meter radii: number of Cafe/Coffee Shops, number of Cafe/Coffee Shops relative to other shops (frequency), number of 20 minute time frames from 6:30 - 8:30 during which Cafe/Coffee Shops are trending, latitude, longitude. These components will be used to compile the k-means clustering sorting.


#### Additional note: Aside from Foursquare data, I'll also consider including publicly available ridership data, depending on 1) the availability of the data, 2) how the data are assembled and 3) whether the data will contribute to clarifying the station data.